In [23]:
import pandas as pd

patients = pd.read_csv('disease.csv')
# Afficher les 10 première lignes du DataFrame.
patients.head(10)
# Afficher les informations des patients ayant 37 ans. Sont-ils tous sains ?
print(patients[patients['age'] == 37])

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
1     37    1   2       130   250    0        1      187      0      3.5   
115   37    0   2       120   215    0        1      170      0      0.0   

     slope  ca  thal  target  
1        0   0     2       1  
115      2   0     2       1  


In [24]:
# Afficher les informations concernant le patient le plus âgé. Est-il malade ?
print(patients[patients['age'] == patients['age'].max()])

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
238   77    1   0       125   304    0        0      162      1      0.0   

     slope  ca  thal  target  
238      2   3     2       0  


In [25]:
# Comparer les proportions de malades et non malades entre les patients de sexe masculin (0) et féminin (1)
print(patients.groupby(['sex', 'target']).size().unstack())

target    0   1
sex            
0        24  72
1       114  93


In [26]:
# Comparer la moyenne d’âge entre les individus sains et les individus malades
print(patients.groupby('target')['age'].mean())

target
0    56.601449
1    52.496970
Name: age, dtype: float64


In [31]:
# Repérer les valeurs aberrantes (c-à-d celles en dehors de l’intervalle [50,250]) de la colonne talach et les remplacer s’il y en a, par une valeur plausible.
patients.loc[(patients['thalach'] < 50) | (patients['thalach'] > 250), 'thalach'] = patients['thalach'].median()


In [32]:
# Afficher le nombre de valeurs manquantes (s’il y en a)  pour chaque colonne du DataFrame.
patients.isnull().sum()


age         0
sex         0
cp          0
trestbps    0
chol        0
fbs         0
restecg     0
thalach     0
exang       0
oldpeak     0
slope       0
ca          0
thal        0
target      0
dtype: int64

In [38]:
# Supprimer les lignes du DataFrame qui ne sont pas labélisées (s’il y en a). Autrement dit, les lignes pour lesquelles la valeur de la colonne target est absente.
print(patients.drop('target', axis=1))

     age  sex  cp  trestbps  chol  fbs  restecg  thalach  exang  oldpeak  \
0     63    1   3       145   233    1        0      150      0      2.3   
1     37    1   2       130   250    0        1      187      0      3.5   
2     41    0   1       130   204    0        0      172      0      1.4   
3     56    1   1       120   236    0        1      178      0      0.8   
4     57    0   0       120   354    0        1      163      1      0.6   
..   ...  ...  ..       ...   ...  ...      ...      ...    ...      ...   
298   57    0   0       140   241    0        1      123      1      0.2   
299   45    1   3       110   264    0        1      132      0      1.2   
300   68    1   0       144   193    1        1      141      0      3.4   
301   57    1   0       130   131    0        1      115      1      1.2   
302   57    0   1       130   236    0        0      174      0      0.0   

     slope  ca  thal  
0        0   0     1  
1        0   0     2  
2        2   0    

In [39]:
# Remplacer les valeurs manquantes des colonnes ‘ca’ et ‘exang’ par leurs modes respectifs.
patients['ca'] = patients['ca'].fillna(patients['ca'].mode()[0])
patients['exang'] = patients['exang'].fillna(patients['exang'].mode()[0])

In [40]:
# Remplacer les valeurs manquantes des autres colonnes par leurs médianes respectives
patients = patients.fillna(patients.median())

In [42]:
# Séparer les variables explicatives dans un DataFrame X et la variable cible dans Y
x = patients.drop('target', axis=1)
y = patients['target']

In [45]:
# Appliquer la normalisation de type Min-Max légèrement modifiée
X_norm = x.apply(lambda x: 2 * (x - x.min()) / (x.max() - x.min()) - 1)
print(X_norm)

          age  sex        cp  trestbps      chol  fbs  restecg   thalach  \
0    0.416667  1.0  1.000000 -0.037736 -0.511416  1.0     -1.0  0.206107   
1   -0.666667  1.0  0.333333 -0.320755 -0.433790 -1.0      0.0  0.770992   
2   -0.500000 -1.0 -0.333333 -0.320755 -0.643836 -1.0     -1.0  0.541985   
3    0.125000  1.0 -0.333333 -0.509434 -0.497717 -1.0      0.0  0.633588   
4    0.166667 -1.0 -1.000000 -0.509434  0.041096 -1.0      0.0  0.404580   
..        ...  ...       ...       ...       ...  ...      ...       ...   
298  0.166667 -1.0 -1.000000 -0.132075 -0.474886 -1.0      0.0 -0.206107   
299 -0.333333  1.0  1.000000 -0.698113 -0.369863 -1.0      0.0 -0.068702   
300  0.625000  1.0 -1.000000 -0.056604 -0.694064  1.0      0.0  0.068702   
301  0.166667  1.0 -1.000000 -0.320755 -0.977169 -1.0      0.0 -0.328244   
302  0.166667 -1.0 -0.333333 -0.320755 -0.497717 -1.0     -1.0  0.572519   

     exang   oldpeak  slope   ca      thal  
0     -1.0 -0.258065   -1.0 -1.0 -0.333333

In [46]:
# Entrainer un modèle de classification de votre choix sur l’ensemble d’entrainement (X_Train, Y_train)
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
# Diviser le jeu de données (X_norm,y) en un ensemble d’entrainement et un ensemble de test
X_train, X_test, y_train, y_test = train_test_split(X_norm, y, test_size=0.3, random_state=42)

model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [47]:

from sklearn.metrics import accuracy_score, confusion_matrix
# Afficher le taux de bonnes prédictions du modèle sur l’échantillon de test, ainsi que la matrice de confusion
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
conf_matrix = confusion_matrix(y_test, y_pred)
print("Accuracy: ", accuracy)
print("Confusion Matrix: \n", conf_matrix)

# Sur l’échantillon de test, combien de patients malades n’ont pas été détectés par le modèle ?
false_negatives = conf_matrix[1][0]
print("Nombre de patients malades non détectés: ", false_negatives)

Accuracy:  0.8021978021978022
Confusion Matrix: 
 [[33  8]
 [10 40]]
Nombre de patients malades non détectés:  10
